# UMAP

In [1]:
import sys
import seaborn as sns
import pandas as pd 
import numpy as np
from itertools import combinations
from scipy.spatial.distance import squareform, pdist
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
import torch
import anndata as ad
import scanpy as sc
import os
import gc
from importlib import reload
print("C1")

from datasets import Dataset, load_from_disk
from datasets import load_dataset
from geneformer import EmbExtractor

from sklearn.cluster import KMeans
print("C2")

#classifer tools
import xgboost
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import confusion_matrix
print("C3")

sns.set_style('white')
torch.cuda.empty_cache()

import os

from datasets import Dataset, load_from_disk, load_dataset
import geneformer

from datetime import datetime
print("C4")
DEFAULT_NAME_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_name_id_dict.pkl"
DEFAULT_TOKEN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/token_dictionary.pkl"
DEFAULT_MEDIAN_PATH = "/nfs/turbo/umms-indikar/shared/projects/geneformer/geneformer/gene_median_dictionary.pkl"

sns.set_style('white')
torch.cuda.empty_cache()
print("C5")

C1
C2
C3
C4
C5


# Importing the anndata files. 

In [2]:
## Iniitla cells
# Just pick from a random file where they were copied (the first half ish of them have them still in original). Then filter by type

harvest_file_path = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/2024-07-31_03-49-01_job_number_22.h5ad'
harvest_file_adata = ad.read_h5ad(harvest_file_path)
initial_adata = harvest_file_adata[harvest_file_adata.obs['type'] == 'initial']


# get adata up and working
# Ensure hsc_adata is already defined and populated with data

# 1. Check the contents of the original obs
print("Original obs contents:")
print(initial_adata.obs.head())

# 2. Create the backup
initial_adata.uns['obs_backup'] = initial_adata.obs.copy()

# 3. Verify the backup was created correctly
print("Contents of obs_backup after creation:")
print(initial_adata.uns['obs_backup'].head())

# 4. Clear the original obs
initial_adata.obs = pd.DataFrame(index=initial_adata.obs.index)

# 5. Check the contents of the cleared obs and the backup
print("Contents of cleared obs:")
print(initial_adata.obs.head())

print("Contents of obs_backup after clearing original obs:")
print(initial_adata.uns['obs_backup'].head())



#initial_adata.obs['kmeans_cluster'] = initial_kmeans.labels_
initial_adata.obs['recipe_name'] = np.nan
initial_adata.obs['recipe_number'] = np.nan
initial_adata.obs['cell_id'] = initial_adata.uns['obs_backup']['cell_id']
initial_adata.obs['type'] = 'initial'



initial_adata.obs.sample(10)



Original obs contents:
                                   input_ids   cell_type         dataset  \
0  [16345  9009 13048 ...  9332 13451  5456]  fibroblast  TS_Vasculature   
1  [12119  9190 16876 ... 11232  1132  1022]  fibroblast  TS_Vasculature   
2  [ 3878  9009  4115 ...  4697 10362 12098]  fibroblast  TS_Vasculature   
3  [ 3878 16916 18367 ... 15470  3946  2153]  fibroblast  TS_Vasculature   
4  [ 6196 16916 10920 ...  7629 10148 17125]  fibroblast  TS_Vasculature   

  length      ignore standardized_cell_type  broad_type __index_level_0__  \
0   2048  fibroblast             Fibroblast  fibroblast            109770   
1   2048  fibroblast             Fibroblast  fibroblast            109771   
2   2048  fibroblast             Fibroblast  fibroblast            109774   
3   2048  fibroblast             Fibroblast  fibroblast            109776   
4   2048  fibroblast             Fibroblast  fibroblast            109777   

  cell_id recipe     type  
0  cell_1    raw  initial  
1

/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_300948/1375448826.py:17: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  initial_adata.uns['obs_backup'] = initial_adata.obs.copy()


,recipe_name,recipe_number,cell_id,type
10412,NaN,NaN,cell_10413,initial
5049,NaN,NaN,cell_5050,initial
10633,NaN,NaN,cell_10634,initial
9362,NaN,NaN,cell_9363,initial
4064,NaN,NaN,cell_4065,initial
3119,NaN,NaN,cell_3120,initial
5670,NaN,NaN,cell_5671,initial
3536,NaN,NaN,cell_3537,initial
13946,NaN,NaN,cell_13947,initial
10993,NaN,NaN,cell_10994,initial


# ISP Cells

In [3]:
# file = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/' + '2024-07-31_03-17-04_job_number_12.h5ad'
# reprog_centroid_adata_wid = ad.read_h5ad(reprog_wid_path)
# # this is not actually written into the file idk if we care? 
# reprog_centroid_adata_wid.obs['type'] = 'reprogrammed'
# reprog_centroid_adata_wid.obs['cluster_number'] = range(1, len(reprog_centroid_adata_wid.obs) + 1)
# kmeans = KMeans(n_clusters=10, random_state=0)

# # Fit KMeans to the DataFrame (make sure to use the feature columns only)
# reprog_centroid_adata['kmeans'] = kmeans.fit_predict(reprog_centroid_adata)

# reprog_centroid_adata_wid.obs.sample(10)


In [4]:
# from itertools import combinations
# import pandas as pd
# import anndata as ad
# import numpy as np
# from sklearn.cluster import KMeans

# # Initialize an empty AnnData object (This will be replaced by actual data)
# reprog_centroid_adata = ad.AnnData()

# all_cluster_labels = []
# all_cell_id_labels = []
# all_recipe_name_labels = []
# all_recipe_num_labels = []

# def ten_choose_five():
#     gene_list = [
#         'GATA2', 
#         'GFI1B', 
#         'FOS', 
#         'STAT5A',
#         'REL',
#         'FOSB',
#         'IKZF1',
#         'RUNX3',
#         'MEF2C',
#         'ETV6',
#     ]
#     len_sublist = 5
#     sublists = list(combinations(gene_list, len_sublist))
    
#     df = pd.DataFrame({
#         'recipe_iteration': range(1, len(sublists) + 1),
#         'recipe_list': [list(sublist) for sublist in sublists]
#     })
#     return df

# pert_df = ten_choose_five()
# pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
# pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# print("Going to read in ")
# print(file)
# one_recipe_adata = ad.read_h5ad(file)
# print(one_recipe_adata.obs.columns)

# one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

# # Perform k-means clustering
# kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
# one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

# # Collect data for the AnnData object
# all_cell_id_labels = one_recipe_adata.obs['cell_id'].tolist()
# all_cluster_labels = one_recipe_adata.obs['kmeans_clusters'].tolist()

# recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
# if len(recipe_names_this_file) == 1:
#     recipe_as_string = recipe_names_this_file[0]
#     recipe_as_list = recipe_as_string.split(';')
#     print(f'The unique value is: {recipe_as_list}')
#     recipe_num = pert_dict[recipe_as_string]
#     print(recipe_num)
# else:
#     print('Error: There are multiple unique values or no values in the column.')

# # Prepare the AnnData object with all cells
# reprog_centroid_adata = ad.AnnData(X=one_recipe_adata.X)
# reprog_centroid_adata.obs['kmeans_cluster'] = all_cluster_labels
# reprog_centroid_adata.obs['cell_id'] = all_cell_id_labels
# reprog_centroid_adata.obs['recipe_name'] = [recipe_as_string] * len(all_cluster_labels)
# reprog_centroid_adata.obs['recipe_number'] = [recipe_num] * len(all_cluster_labels)

# print("All cell data and kmeans_cluster values have been added to reprog_centroid_adata.")
# print(reprog_centroid_adata.obs.head(12))


In [5]:
# from itertools import combinations
# import pandas as pd
# import anndata as ad
# import numpy as np
# from sklearn.cluster import KMeans
# import os

# def ten_choose_five():
#     gene_list = [
#         'GATA2', 
#         'GFI1B', 
#         'FOS', 
#         'STAT5A',
#         'REL',
#         'FOSB',
#         'IKZF1',
#         'RUNX3',
#         'MEF2C',
#         'ETV6',
#     ]
#     len_sublist = 5
#     sublists = list(combinations(gene_list, len_sublist))
    
#     df = pd.DataFrame({
#         'recipe_iteration': range(1, len(sublists) + 1),
#         'recipe_list': [list(sublist) for sublist in sublists]
#     })
#     return df

# pert_df = ten_choose_five()
# pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
# pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# # List of files to process
# file_list = ['/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-03-23_job_number_89.h5ad','/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_07-01-47_job_number_88.h5ad']  # Replace with actual file names

# # Initialize a list to collect data
# all_data = []

# for file in file_list:
#     print("Going to read in ", file)
#     one_recipe_adata = ad.read_h5ad(file)
#     print(one_recipe_adata.obs.columns)

#     one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

#     # Perform K-means clustering
#     kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
#     one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

#     # Extract recipe number
#     recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
#     if len(recipe_names_this_file) == 1:
#         recipe_as_string = recipe_names_this_file[0]
#         recipe_num = pert_dict.get(recipe_as_string, None)
#         if recipe_num is None:
#             print(f"Recipe number not found for {recipe_as_string}")
#             continue
#     else:
#         print('Error: There are multiple unique values or no values in the column.')
#         continue

#     # Combine recipe number with kmeans cluster number
#     one_recipe_adata.obs['kmeans'] = one_recipe_adata.obs['kmeans_clusters'].apply(
#         lambda x: int(f"{recipe_num}{x}")
#     )

#     # Collect data
#     all_data.append(one_recipe_adata)

# # Concatenate all AnnData objects
# combined_adata = ad.concat(all_data, join='outer')

# print("Cluster counts:")
# combined_adata.obs.sample(20)

# # she works b"h


In [6]:
# ok here's where we have to be careful.  going back to an old version of the file to find where reprog_centroid_adata_wid was made, and if there's composite of the original cells. checking in turbo first


In [7]:
import os
import re

def list_reprogramming_jobs(directory, start_job_number, end_job_number):
    """
    List and sort files in the given directory by job number within the specified range.

    Parameters:
    directory (str): The directory containing the files.
    start_job_number (int): The starting job number.
    end_job_number (int): The ending job number.

    Returns:
    list: A sorted list of file names within the specified job number range.
    """
    # List all files in the directory
    files = os.listdir(directory)

    # Define a regex pattern to extract job numbers
    pattern = re.compile(r'_job_number_(\d+)\.h5ad$')

    # Create a list of tuples (job_number, file_name)
    files_with_job_numbers = []
    for file in files:
        match = pattern.search(file)
        if match:
            job_number = int(match.group(1))
            if start_job_number <= job_number <= end_job_number:
                files_with_job_numbers.append((job_number, file))

    # Sort files by job number
    files_with_job_numbers.sort(key=lambda x: x[0])

    # Extract sorted file names
    sorted_files = [file for _, file in files_with_job_numbers]

    return sorted_files



directory = '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/'
files_in_directory = list_reprogramming_jobs(directory, 1, 252)
files_in_directory = [os.path.join(directory,file) for file in files_in_directory]
files_in_directory

['/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-44_job_number_1.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-45_job_number_2.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-53_job_number_3.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-49_job_number_4.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-45-28_job_number_5.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-55_job_number_6.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-45-23_job_number_7.h5ad',
 '/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_02-44-57_job_number_8.h5ad',
 '/nfs/turbo/umm

In [8]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
from itertools import combinations
import pandas as pd
import anndata as ad
import numpy as np
from sklearn.cluster import KMeans
import os

def ten_choose_five():
    gene_list = [
        'GATA2', 
        'GFI1B', 
        'FOS', 
        'STAT5A',
        'REL',
        'FOSB',
        'IKZF1',
        'RUNX3',
        'MEF2C',
        'ETV6',
    ]
    len_sublist = 5
    sublists = list(combinations(gene_list, len_sublist))
    
    df = pd.DataFrame({
        'recipe_iteration': range(1, len(sublists) + 1),
        'recipe_list': [list(sublist) for sublist in sublists]
    })
    return df

pert_df = ten_choose_five()
pert_df['recipe_list'] = pert_df['recipe_list'].apply(lambda x: ';'.join(x))
pert_dict = pert_df.set_index('recipe_list')['recipe_iteration'].to_dict()

# List of files to process
  # Replace with actual file names

# Initialize a list to collect data
all_data = []

for file in files_in_directory:
    print("Going to read in ", file)
    one_recipe_adata = ad.read_h5ad(file)
    print(one_recipe_adata.obs.columns)

    one_recipe_adata = one_recipe_adata[one_recipe_adata.obs['type'] == 'reprogrammed']

    # Perform K-means clustering
    kmeans = KMeans(n_clusters=10, random_state=0).fit(one_recipe_adata.X)
    one_recipe_adata.obs['kmeans_clusters'] = kmeans.labels_

    # Extract recipe number
    recipe_names_this_file = one_recipe_adata.obs['recipe'].unique()
    if len(recipe_names_this_file) == 1:
        recipe_as_string = recipe_names_this_file[0]
        recipe_num = pert_dict.get(recipe_as_string, None)
        if recipe_num is None:
            print(f"Recipe number not found for {recipe_as_string}")
            continue
    else:
        print('Error: There are multiple unique values or no values in the column.')
        continue

    # Combine recipe number with kmeans cluster number
    one_recipe_adata.obs['kmeans'] = one_recipe_adata.obs['kmeans_clusters'].apply(
        lambda x: int(f"{recipe_num}{x}")
    )

    # Collect data
    all_data.append(one_recipe_adata)

# Concatenate all AnnData objects
combined_adata = ad.concat(all_data, join='outer')

# she works b"h


In [ ]:
# and we pray 
combined_adata.obs.sample(20)

In [ ]:


# youve gotta delete the duplicate 252 again girlpop
# Assuming `combined_adata` is your AnnData object

# Get the rows where 'kmeans_clusters' is equal to 252
rows_with_252 = (combined_adata.obs['kmeans'] >= 2520)

# Count the number of rows with kmeans_clusters equal to 252
num_rows_252 = rows_with_252.sum()

print(f"Number of rows with 'kmeans' equal to 252: {num_rows_252}")


In [ ]:
mask = (combined_adata.obs['kmeans'] >= 1830) | (combined_adata.obs['kmeans'] <= 1839) 
# Filter the DataFrame based on the mask
test_obs = combined_adata.obs[mask]

# Step 2: Count the number of unique rows
num_unique_rows = test_obs.shape[0]
num_unique_rows

In [ ]:
# get rid of all rows with kmeans >= 2520 doesn't solve it. the other dataframe with < 2520 is a whack job. on the one hand,
# it seems like each bin (binned together by recipe) has 15308 in it with the exception of 0-9 bc we started cluster number at 10 
# instead of 00.
# that actually doesnt fix it we conditioned fine


In [ ]:
# import pandas as pd

# # Assuming `combined_adata` is your AnnData object and 'relevant_column' is the column to filter

# # Step 1: Create masks for rows based on the condition
# mask_less_equal = combined_adata.obs['kmeans'] < 2520
# mask_greater = combined_adata.obs['kmeans'] >= 2520

# # Filter the DataFrame based on these masks
# filtered_obs_less_equal = combined_adata.obs[mask_less_equal]
# filtered_obs_greater = combined_adata.obs[mask_greater]

# # Step 2: Remove duplicate rows from those with values greater than 2520
# unique_filtered_obs_greater = filtered_obs_greater.drop_duplicates()

# # Combine the two filtered DataFrames
# combined_filtered_obs = pd.concat([filtered_obs_less_equal, unique_filtered_obs_greater])

# # Step 3: Update `combined_adata` with the combined filtered DataFrame
# combined_adata_no_dupe = combined_adata[combined_adata.obs.index.isin(combined_filtered_obs.index)]

# # Print the number of remaining rows
# print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")
# # 3872924 is (252*15308) + 15309 extra from the extra 252 file.


In [ ]:
# import pandas as pd

# # Assuming `combined_adata` is your AnnData object and 'kmeans' is the column to filter

# # Step 1: Create masks for rows based on the condition
# mask_less_equal = combined_adata.obs['kmeans'] < 2520
# mask_greater = combined_adata.obs['kmeans'] >= 2520

# # Filter the DataFrame based on these masks
# filtered_obs_less_equal = combined_adata.obs[mask_less_equal]
# filtered_obs_greater = combined_adata.obs[mask_greater]

# # Step 2: Remove duplicate rows from those with values greater than 2520
# # Consider dropping duplicates based on all columns
# unique_filtered_obs_greater = filtered_obs_greater.drop_duplicates()

# # Print the number of rows before and after dropping duplicates
# print(f"Number of rows before dropping duplicates: {filtered_obs_greater.shape[0]}")
# print(f"Number of rows after dropping duplicates: {unique_filtered_obs_greater.shape[0]}")

# # Combine the two filtered DataFrames
# combined_filtered_obs = pd.concat([filtered_obs_less_equal, unique_filtered_obs_greater])

# # Step 3: Update `combined_adata` with the combined filtered DataFrame
# combined_adata_no_dupe = combined_adata[combined_adata.obs.index.isin(combined_filtered_obs.index)]

# # Print the number of remaining rows
# print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")

# # Additional check: Print the counts of 'kmeans' values before and after filtering
# print("Counts before filtering:")
# print(combined_adata.obs['kmeans'].value_counts())

# print("Counts after filtering:")
# print(combined_adata_no_dupe.obs['kmeans'].value_counts())


In [ ]:
# import pandas as pd
# import numpy as np

# # Assuming `combined_adata` is your AnnData object and 'kmeans' is the column to filter

# # Step 1: Create masks for rows based on the condition
# filtered_less = combined_adata[combined_adata.obs['kmeans'] < 2520]
# greater_equal = combined_adata[combined_adata.obs['kmeans'] >= 2520]

# # # Filter the DataFrame based on these masks
# # filtered_obs_less_equal = combined_adata.obs[mask_less_equal]
# # filtered_obs_greater = combined_adata.obs[mask_greater]
# print("C1")

# # Step 2: Remove duplicate rows from those with values greater than 2520
# filtered_greater = greater_equal[~greater_equal.obs.duplicated(keep='first')]
# #filtered_greater.X
# filtered_less.X.shape[0]
# #combined_adata_no_dupe = ad.concat([filtered_less, filtered_greater], join='outer')
# # print("C2")
# # # Create a boolean mask to select rows from `combined_adata` that are either:
# # # - Less than 2520 (no duplicates) 
# # # - Greater than or equal to 2520 and are unique
# # # mask_final = np.concatenate([
# # #     mask_less_equal.values,  # All rows where 'kmeans' < 2520
# # #     np.isin(combined_adata.obs.index, unique_filtered_obs_greater.index)  # Unique rows where 'kmeans' >= 2520
# # # ])
# # mask_final = m
# # print("C3")
# # # Step 3: Filter the AnnData object based on the combined mask
# # combined_adata_no_dupe = combined_adata[mask_final]

# # # Print the number of remaining rows
# # print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")
# # print("C4")
# # # Additional check: Print the counts of 'kmeans' values before and after filtering
# # print("Counts before filtering:")
# # print(combined_adata.obs['kmeans'].value_counts())

# # print("Counts after filtering:")
# # print(combined_adata_no_dupe.obs['kmeans'].value_counts())
# # print("C5")

In [ ]:
# import pandas as pd

# # Assuming `filtered_less` is the DataFrame or AnnData object you want to analyze

# # Create bins with ranges of 10
# bins = range(0, filtered_less.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(filtered_less.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Check if any bin does not have exactly 15,308 counts
# target_count = 15308
# bins_with_different_count = binned_counts[binned_counts != target_count]

# # Print bins that do not have exactly 15,308 counts
# print(f"Total number of bins: {len(binned_counts)}")
# if not bins_with_different_count.empty:
#     print("Bins with counts different from 15,308:")
#     print(bins_with_different_count)
# else:
#     print("All bins have exactly 15,308 counts.")


In [ ]:
# import pandas as pd

# # Assuming `filtered_less` is the DataFrame or AnnData object you want to analyze

# # Create bins with ranges of 10
# bins = range(0, filtered_less.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(filtered_less.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Print the count in each bin
# print("Counts in each bin:")
# print(binned_counts)

# # Check if any bin does not have exactly 15,308 counts
# target_count = 15308
# bins_with_different_count = binned_counts[binned_counts != target_count]

# # Print bins that do not have exactly 15,308 counts
# print(f"Total number of bins: {len(binned_counts)}")
# if not bins_with_different_count.empty:
#     print("Bins with counts different from 15,308:")
#     print(bins_with_different_count)
# else:
#     print("All bins have exactly 15,308 counts.")


In [ ]:
# import pandas as pd
# import numpy as np

# # Assuming `combined_adata` is your AnnData object and 'kmeans' is the column to filter

# # Step 1: Create masks for rows based on the condition
# mask_less_equal = combined_adata.obs['kmeans'] < 2520
# mask_greater = combined_adata.obs['kmeans'] >= 2520

# # Filter the DataFrame based on these masks
# filtered_obs_greater = combined_adata.obs[mask_greater]

# # Step 2: Remove duplicate rows from those with values greater than 2520
# unique_filtered_obs_greater = filtered_obs_greater.drop_duplicates()

# # Convert the index of unique_filtered_obs_greater to a boolean mask
# unique_mask = combined_adata.obs.index.isin(unique_filtered_obs_greater.index)

# # Combine the masks for filtering
# final_mask = mask_less_equal | unique_mask

# # Step 3: Filter the AnnData object based on the combined mask
# combined_adata_no_dupe = combined_adata[final_mask]

# # Print the number of remaining rows
# print(f"Number of remaining rows: {combined_adata_no_dupe.n_obs}")

# # Additional check: Print the counts of 'kmeans' values before and after filtering
# print("Counts before filtering:")
# print(combined_adata.obs['kmeans'].value_counts())

# print("Counts after filtering:")
# print(combined_adata_no_dupe.obs['kmeans'].value_counts())


In [ ]:
# import pandas as pd

# # Assuming `combined_adata` is your AnnData object

# # Create bins with ranges of 10
# bins = range(0, combined_adata_no_dupe.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(combined_adata_no_dupe.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Print the counts for each bin
# print(binned_counts)

# import pandas as pd

# # Assuming `combined_adata` is your AnnData object

# # Create bins with ranges of 10
# bins = range(0, combined_adata_no_dupe.obs['kmeans'].max() + 10, 10)

# # Create labels for each bin
# bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# # Use pd.cut to categorize 'kmeans' values into bins
# binned_kmeans = pd.cut(combined_adata_no_dupe.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# # Count occurrences in each bin
# binned_counts = binned_kmeans.value_counts().sort_index()

# # Check if any bin does not have exactly 15,308 counts
# target_count = 15308
# bins_with_different_count = binned_counts[binned_counts != target_count]

# # Print bins that do not have exactly 15,308 counts
# if not bins_with_different_count.empty:
#     print("Bins with counts different from 15,308:")
#     print(bins_with_different_count)
# else:
#     print("All bins have exactly 15,308 counts.")


In [ ]:
# # # Assuming `combined_adata` is your AnnData object

# # # Get the unique values and their counts in the 'kmeans' column
# # kmeans_counts = combined_adata.obs['kmeans'].value_counts()

# # # Print the unique values and their counts
# # print(kmeans_counts)
# # # Assuming `combined_adata` is your AnnData object

# # # Calculate the average value in the 'kmeans' column
# # average_kmeans = combined_adata.obs['kmeans'].mean()

# # # Print the average value
# # print(f"Average value of 'kmeans': {average_kmeans}")
# # Assuming `combined_adata` is your AnnData object

# # Filter for 'kmeans' values between 2520 and 2529
# kmeans_range = combined_adata_.obs['kmeans'][(combined_adata.obs['kmeans'] >= 2520) & (combined_adata.obs['kmeans'] <= 2529)]

# # Get the count of each 'kmeans' value within the range
# kmeans_counts = kmeans_range.value_counts().sort_index()

# # Print the counts for each 'kmeans' value in the range 2520 to 2529
# print(kmeans_counts)
# # sum to 30616 so that's good

In [ ]:
# # writing into turbo
# combined_adata = combined_adata[combined_adata.obs['type'] == 'reprogrammed']
# combined_adata.obs.sample(10)
# combined_adata.write('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/perturbed_combined.h5ad')

# I'm just going to pretend like this isn't an issue for now.

In [9]:
combined_adata.X.shape

NameError: name 'combined_adata' is not defined

In [10]:
import pandas as pd
import numpy as np

# Assuming combined_adata is your AnnData object and 'kmeans' is the column to analyze

# Get the minimum and maximum values
min_value = combined_adata.obs['kmeans'].min()
max_value = combined_adata.obs['kmeans'].max()

# Define the bin edges with a width of 10
bins = np.arange(min_value, max_value + 10, 10)

# Define bin labels
bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# Categorize 'kmeans' values into bins
binned_kmeans = pd.cut(combined_adata.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# Count occurrences in each bin
binned_counts = binned_kmeans.value_counts().sort_index()

# Prepare the results as an array of intervals and their counts
intervals_counts = np.array(list(zip(binned_counts.index, binned_counts.values)))

# Print the intervals and counts
print("Intervals and their counts:")
print(intervals_counts)


NameError: name 'combined_adata' is not defined

In [ ]:
import pandas as pd
import numpy as np

# Assuming combined_adata is your AnnData object and 'kmeans' is the column to analyze

# Get the minimum and maximum values
min_value = combined_adata.obs['kmeans'].min()
max_value = combined_adata.obs['kmeans'].max()

# Define the bin edges with a width of 10
bins = np.arange(min_value, max_value + 10, 10)

# Define bin labels
bin_labels = [f"{i}-{i+9}" for i in bins[:-1]]

# Categorize 'kmeans' values into bins
binned_kmeans = pd.cut(combined_adata.obs['kmeans'], bins=bins, labels=bin_labels, right=False)

# Count occurrences in each bin
binned_counts = binned_kmeans.value_counts().sort_index()

# Prepare the results as an array of intervals and their counts
intervals_counts = np.array(list(zip(binned_counts.index, binned_counts.values)))

# Calculate the sum of the second column (counts)
sum_of_counts = intervals_counts[:, 1].astype(int).sum()

# Print the sum of the counts
print(f"Sum of counts in all bins: {sum_of_counts}")


In [ ]:
# separating out into problem part and not problem part:
import pandas as pd

# Define the bin label for the high bin
high_bin_label = '2520-2529'

# Create a mask for rows in the '2520-2529' bin
high_bin_mask = binned_kmeans == high_bin_label

# Create the high and low DataFrames based on the mask
condensed_adata_high = combined_adata[high_bin_mask]
condensed_adata_low = combined_adata[~high_bin_mask]

# Print the number of rows in each DataFrame to verify
print(f"Number of rows in condensed_adata_high: {condensed_adata_high.n_obs}")
print(f"Number of rows in condensed_adata_low: {condensed_adata_low.n_obs}")


In [ ]:
import pandas as pd
import numpy as np
import anndata

def remove_duplicate_obs(adata):
    # Identify duplicate indices
    duplicated_indices = adata.obs.index.duplicated(keep='first')
    
    # Remove duplicate observations
    adata = adata[~duplicated_indices].copy()
    
    return adata

condensed_adata_high = remove_duplicate_obs(condensed_adata_high)
#condensed_adata_high.n_obs
condensed_adata_high

In [ ]:
combined_adata_no_dupe_true = ad.concat([condensed_adata_low, condensed_adata_high])
combined_adata_no_dupe_true.obs['kmeans'] -= 10
combined_adata_no_dupe_true.obs.tail(10)

In [ ]:
# what you see above is what's in the file.
combined_adata_no_dupe_true.write('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/perturbed_combined_good.h5ad')


In [ ]:
combined_adata_no_dupe_true = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/perturbed_combined_good.h5ad')

In [ ]:
combined_adata_no_dupe_true.obs.shape
# this is the right shape!

# New approach

#### Loading this all in is too hard. let's just take clusters directly from the files we need.

In [ ]:
#'/nfs/turbo/umms-indikar/shared/projects/geneformer/8_14_originals_working/fib15k/2024-07-31_05-34-28_job_number_61.h5ad'
#cluster 7 when 0-9
# pick some rando hsc cells idrc/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/

In [11]:
#initial_adata is the fibroblasts
hsc_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/hsc_1_use_this_copy.h5ad')
max_min_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_05-34-28_job_number_61.h5ad')
max_min_adata = max_min_adata[max_min_adata.obs['type'] == 'reprogrammed']
mean_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_06-34-03_job_number_77.h5ad')
mean_adata = mean_adata[mean_adata.obs['type'] == 'reprogrammed']

/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/oliven/miniconda3/envs/geneformer2/lib/python3.10/site-packages/anndata/_core/aligned_df.py:67: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [12]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0)
max_min_adata.obs['kmeans_clusters'] = kmeans.fit_predict(max_min_adata.X)

mean_adata.obs['kmeans_clusters'] = kmeans.fit_predict(mean_adata.X)

mean_adata.obs.head()


/tmp/ipykernel_300948/703621888.py:4: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  max_min_adata.obs['kmeans_clusters'] = kmeans.fit_predict(max_min_adata.X)
/tmp/ipykernel_300948/703621888.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  mean_adata.obs['kmeans_clusters'] = kmeans.fit_predict(mean_adata.X)


,input_ids,cell_type,dataset,length,ignore,standardized_cell_type,broad_type,__index_level_0__,cell_id,recipe,type,kmeans_clusters
15308,"[14409, 12698, 10804, 15641, 1532, 16345, 9009...",fibroblast,TS_Vasculature,2048,fibroblast,Fibroblast,fibroblast,109770,cell_1,GATA2;FOS;REL;IKZF1;MEF2C,reprogrammed,4
15309,"[14409, 12698, 10804, 15641, 1532, 12119, 9190...",fibroblast,TS_Vasculature,2048,fibroblast,Fibroblast,fibroblast,109771,cell_2,GATA2;FOS;REL;IKZF1;MEF2C,reprogrammed,8
15310,"[14409, 12698, 10804, 15641, 1532, 3878, 9009,...",fibroblast,TS_Vasculature,2048,fibroblast,Fibroblast,fibroblast,109774,cell_3,GATA2;FOS;REL;IKZF1;MEF2C,reprogrammed,8
15311,"[14409, 12698, 10804, 15641, 1532, 3878, 16916...",fibroblast,TS_Vasculature,2048,fibroblast,Fibroblast,fibroblast,109776,cell_4,GATA2;FOS;REL;IKZF1;MEF2C,reprogrammed,0
15312,"[14409, 12698, 10804, 15641, 1532, 6196, 16916...",fibroblast,TS_Vasculature,2048,fibroblast,Fibroblast,fibroblast,109777,cell_5,GATA2;FOS;REL;IKZF1;MEF2C,reprogrammed,9


In [13]:
from sklearn.cluster import KMeans
np.random.seed(23)


plot_max_adata = max_min_adata[(max_min_adata.obs['kmeans_clusters'] == 6)] #this is not arbitrary
plot_min_adata = max_min_adata[(max_min_adata.obs['kmeans_clusters'] == 8)]
random_indices_max = np.random.choice(plot_max_adata.shape[0], 150, replace=False)
random_indices_min = np.random.choice(plot_min_adata.shape[0], 150, replace=False) 
plot_max_adata = plot_max_adata[random_indices_max]
plot_min_adata = plot_min_adata[random_indices_min]

plot_mean_adata = mean_adata[mean_adata.obs['kmeans_clusters'] == 1] #this is arbitrary
random_indices_mean = np.random.choice(plot_mean_adata.shape[0], 300, replace=False)
plot_mean_adata = plot_mean_adata[random_indices_mean]

plot_min_adata.obs.shape

(150, 12)

In [ ]:
np.random.seed(23)

random_indices_fib = np.random.choice(initial_adata.shape[0], 300, replace=False)
plot_fib_adata = initial_adata[random_indices_fib]
plot_fib_adata.obs.shape

In [ ]:
np.random.seed(23)

random_indices_hsc = np.random.choice(hsc_adata.shape[0], 300, replace=False)
plot_hsc_adata = hsc_adata[random_indices_hsc]
plot_hsc_adata.obs.shape

In [ ]:
# #clearing out obs

plot_max_adata.obs = pd.DataFrame(index=plot_max_adata.obs.index)
plot_min_adata.obs = pd.DataFrame(index=plot_min_adata.obs.index)

plot_max_adata.obs['dataset'] = "max_61_7"
plot_min_adata.obs['dataset'] = "min_61_9"

plot_mean_adata.obs = pd.DataFrame(index=plot_mean_adata.obs.index)
plot_mean_adata.obs['dataset'] = "mean_77_2"

plot_hsc_adata.obs = pd.DataFrame(index=plot_hsc_adata.obs.index)
plot_hsc_adata.obs['dataset'] = "hsc_hsc1_NaN"

plot_fib_adata.obs = pd.DataFrame(index=plot_fib_adata.obs.index)
plot_fib_adata.obs['dataset'] = "fib_NaN_NaN"

plot_adata = ad.concat([plot_max_adata, plot_min_adata, plot_mean_adata, plot_fib_adata, plot_hsc_adata])
plot_adata.obs.sample(15)


In [ ]:
import umap
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Set up the UMAP reducer
reducer = umap.UMAP(random_state=42)

# Fit and transform the data
umap_embedding = reducer.fit_transform(plot_adata.X)

# Create a new DataFrame for easy plotting
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset'].values

# Define custom colors with valid color names or hex codes
colors = {
    'max_61_7': '#301934',    # Dark purple
    'min_61_9': '#FF00FF',     # Magenta
    'mean_77_2': '#E6E6FA',   # Lavender
    'hsc_hsc1_NaN': '#FF007F', # Bright pink
    'fib_NaN_NaN': '#0000FF'   # Blue
}

# Map the colors to the dataset column
umap_df['color'] = umap_df['dataset'].map(colors)

# Plot the UMAP
plt.figure(figsize=(10, 8))
sns.scatterplot(
    data=umap_df,
    x='UMAP1', y='UMAP2',
    hue='dataset',
    palette=colors,
    s=20,  # Size of points
    alpha=0.8  # Transparency for better visibility of overlapping points
)

# Set the title and labels
plt.title('UMAP of plot_adata Vectors')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

# Show the plot
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

datasets = ['max_61_7', 'min_61_9', 'mean_77_2', 'fib_NaN_NaN', 'hsc_hsc1_NaN']
centroids_high_dim = {dataset: calculate_centroids(plot_adata, dataset) for dataset in datasets}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array(list(centroids_high_dim.values()))
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Plot the UMAP
plt.figure(figsize=(12, 8))
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette='RdBu', alpha=0.6)

# Add centroid markers
centroid_labels = datasets
for i, dataset in enumerate(datasets):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], marker='X', s=100, label=f'Centroid {dataset}', edgecolor='black', color='red')

# 4. Draw Lines Between Centroids and Compute Cosine Distances
distances = cosine_distances(centroids_high_dim_matrix)
for i, start in enumerate(datasets):
    for j, end in enumerate(datasets):
        if i < j:
            plt.plot([centroids_umap[i, 0], centroids_umap[j, 0]], [centroids_umap[i, 1], centroids_umap[j, 1]], 'k--', alpha=0.6)
            mid_point = [(centroids_umap[i, 0] + centroids_umap[j, 0]) / 2, (centroids_umap[i, 1] + centroids_umap[j, 1]) / 2]
            plt.text(mid_point[0], mid_point[1], f'{distances[i, j]:.2f}', color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array(list(centroids_high_dim.values()))
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Plot the UMAP
plt.figure(figsize=(12, 8))
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette='RdBu', alpha=0.6)

# Add centroid markers for the combined dataset and others
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], marker='X', s=100, label=f'Centroid {dataset}', edgecolor='black', color='red')

# 4. Draw Lines Between Centroids and Compute Cosine Distances
distances = cosine_distances(centroids_high_dim_matrix)
for i, start in enumerate(centroid_labels):
    for j, end in enumerate(centroid_labels):
        if i < j:
            plt.plot([centroids_umap[i, 0], centroids_umap[j, 0]], [centroids_umap[i, 1], centroids_umap[j, 1]], 'k--', alpha=0.6)
            mid_point = [(centroids_umap[i, 0] + centroids_umap[j, 0]) / 2, (centroids_umap[i, 1] + centroids_umap[j, 1]) / 2]
            plt.text(mid_point[0], mid_point[1], f'{distances[i, j]:.2f}', color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each centroid
centroid_colors = {
    'max_min_combined': 'green',
    'mean_77_2': '#B2182B',  # color from RdBu palette
    'fib_NaN_NaN': '#2166AC',  # color from RdBu palette
    'hsc_hsc1_NaN': '#F4A582'  # color from RdBu palette
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array(list(centroids_high_dim.values()))
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Plot the UMAP
plt.figure(figsize=(12, 8))
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette='RdBu', alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=100, label=f'Centroid {dataset}', 
                edgecolor='black', color=centroid_colors[dataset])

# 4. Draw Lines Between Centroids and Compute Cosine Distances
distances = cosine_distances(centroids_high_dim_matrix)
for i, start in enumerate(centroid_labels):
    for j, end in enumerate(centroid_labels):
        if i < j:
            plt.plot([centroids_umap[i, 0], centroids_umap[j, 0]], 
                     [centroids_umap[i, 1], centroids_umap[j, 1]], 
                     'k--', alpha=0.6)
            mid_point = [(centroids_umap[i, 0] + centroids_umap[j, 0]) / 2, 
                         (centroids_umap[i, 1] + centroids_umap[j, 1]) / 2]
            plt.text(mid_point[0], mid_point[1], f'{distances[i, j]:.2f}', 
                     color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'max_61_7': centroid_max,
    'min_61_9': centroid_min,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each dataset
centroid_colors = {
    'max_61_7': 'blue',  # Example color
    'min_61_9': 'orange',  # Example color
    'mean_77_2': '#E41A1C',
    'fib_NaN_NaN': '#377EB8',
    'hsc_hsc1_NaN': '#FF7F00',
    'max_min_combined': 'green'
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array(list(centroids_high_dim.values()))
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Plot the UMAP
plt.figure(figsize=(12, 8))

# Plot the UMAP scatter plot with the updated palette
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette=centroid_colors, alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=100, label=f'Centroid {dataset}', 
                edgecolor='black', color=centroid_colors[dataset])

# 4. Draw Lines Between Centroids and Compute Cosine Distances
distances = cosine_distances(centroids_high_dim_matrix)
for i, start in enumerate(centroid_labels):
    for j, end in enumerate(centroid_labels):
        if i < j:
            plt.plot([centroids_umap[i, 0], centroids_umap[j, 0]], 
                     [centroids_umap[i, 1], centroids_umap[j, 1]], 
                     'k--', alpha=0.6)
            mid_point = [(centroids_umap[i, 0] + centroids_umap[j, 0]) / 2, 
                         (centroids_umap[i, 1] + centroids_umap[j, 1]) / 2]
            plt.text(mid_point[0], mid_point[1], f'{distances[i, j]:.2f}', 
                     color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
# Heatmap between mean_61_7 and hsc_hsc1_NaN datasets

In [ ]:

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Extract the relevant data points
mean_data = plot_adata[plot_adata.obs['dataset'] == 'mean_77_2'].X
hsc_data = plot_adata[plot_adata.obs['dataset'] == 'hsc_hsc1_NaN'].X

# Compute pairwise distances between the datasets
pairwise_distances = cdist(mean_data, hsc_data, metric='euclidean')

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(pairwise_distances, cmap='viridis')
plt.title('Pairwise Distances between mean_61_7 and hsc_hsc1_NaN Datasets')
plt.xlabel('hsc_hsc1_NaN Points')
plt.ylabel('mean_77_2 Points')
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import linkage, leaves_list

# List of datasets
datasets = ['fib_NaN_NaN', 'hsc_hsc1_NaN', 'max_61_7', 'mean_77_2']

# Initialize a list to hold data and labels
data_matrices = []
labels = []

# Extract data points for each dataset and store them
for dataset in datasets:
    data = plot_adata[plot_adata.obs['dataset'] == dataset].X
    data_matrices.append(data)
    labels.extend([dataset] * data.shape[0])

# Compute pairwise distances between all data points across datasets
pairwise_distances = cdist(np.vstack(data_matrices), np.vstack(data_matrices), metric='euclidean')

# Perform hierarchical clustering to reorder datasets
linked = linkage(pairwise_distances, method='average')
order = leaves_list(linked)

# Reorder the distance matrix and labels according to the clustering
ordered_distances = pairwise_distances[order, :][:, order]
ordered_labels = np.array(labels)[order]

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(ordered_distances, cmap='viridis', xticklabels=ordered_labels, yticklabels=ordered_labels)
plt.title('Pairwise Distances between Datasets')
plt.xlabel('Data Points')
plt.ylabel('Data Points')
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import linkage, leaves_list

# List of datasets
datasets = ['fib_NaN_NaN', 'hsc_hsc1_NaN', 'max_61_7', 'mean_77_2']

# Initialize a list to hold data and labels
data_matrices = []
dataset_labels = []

# Extract data points for each dataset and store them
for dataset in datasets:
    data = plot_adata[plot_adata.obs['dataset'] == dataset].X
    data_matrices.append(data)
    dataset_labels.extend([dataset] * data.shape[0])

# Compute pairwise distances between all data points across datasets
pairwise_distances = cdist(np.vstack(data_matrices), np.vstack(data_matrices), metric='euclidean')

# Perform hierarchical clustering to reorder datasets
linked = linkage(pairwise_distances, method='average')
order = leaves_list(linked)

# Reorder the distance matrix and dataset labels according to the clustering
ordered_distances = pairwise_distances[order, :][:, order]
ordered_dataset_labels = np.array(dataset_labels)[order]

# Map minor labels to dataset names
unique_datasets = np.unique(ordered_dataset_labels)
axis_labels = []
for label in ordered_dataset_labels:
    for dataset in unique_datasets:
        if label == dataset:
            axis_labels.append(dataset)
            break

# Plot the heatmap without minor labels
plt.figure(figsize=(12, 10))
sns.heatmap(ordered_distances, cmap='viridis', xticklabels=False, yticklabels=False)
plt.title('Pairwise Distances between Datasets')

# Set major ticks at the centers of each block of data points and label with dataset names
num_points = [np.sum(ordered_dataset_labels == dataset) for dataset in unique_datasets]
positions = np.cumsum([0] + num_points) - 0.5 * np.array(num_points)

plt.xticks(positions, unique_datasets, rotation=45, ha='right')
plt.yticks(positions, unique_datasets, rotation=0)

plt.xlabel('Dataset')
plt.ylabel('Dataset')
plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist
from scipy.cluster.hierarchy import linkage, leaves_list
from scipy.spatial.distance import squareform

# List of datasets
datasets = ['fib_NaN_NaN', 'hsc_hsc1_NaN', 'max_61_7', 'mean_77_2']

# Initialize a list to hold data and labels
data_matrices = []
dataset_labels = []

# Extract data points for each dataset and store them
for dataset in datasets:
    data = plot_adata[plot_adata.obs['dataset'] == dataset].X
    data_matrices.append(data)
    dataset_labels.extend([dataset] * data.shape[0])

# Compute pairwise distances between all data points across datasets
pairwise_distances = cdist(np.vstack(data_matrices), np.vstack(data_matrices), metric='euclidean')

# Convert the full distance matrix to a condensed distance matrix
condensed_distances = squareform(pairwise_distances)

# Perform hierarchical clustering to reorder datasets
linked = linkage(condensed_distances, method='average')
order = leaves_list(linked)

# Reorder the distance matrix and dataset labels according to the clustering
ordered_distances = pairwise_distances[order, :][:, order]
ordered_dataset_labels = np.array(dataset_labels)[order]

# Compute the number of points per dataset
unique_datasets, counts = np.unique(ordered_dataset_labels, return_counts=True)
num_datasets = len(unique_datasets)
positions = np.concatenate([[0], np.cumsum(counts)])

# Define positions for the ticks based on dataset block positions
tick_positions = positions[:-1] + np.diff(positions) / 2

# Plot the heatmap
plt.figure(figsize=(12, 10))
ax = sns.heatmap(ordered_distances, cmap='viridis', xticklabels=False, yticklabels=False)

# Set major ticks and labels
ax.set_xticks(tick_positions)
ax.set_xticklabels(unique_datasets, rotation=45, ha='right')
ax.set_yticks(tick_positions)
ax.set_yticklabels(unique_datasets, rotation=0)

plt.title('Pairwise Distances between Datasets')
plt.xlabel('Dataset')
plt.ylabel('Dataset')
plt.show()


## Fine Tuning the Distance UMAP

In [ ]:
#initial_adata is the fibroblasts
hsc_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/hsc_1_use_this_copy.h5ad')
max_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_05-34-28_job_number_61.h5ad')
max_adata = max_adata[max_adata.obs['type'] == 'reprogrammed']
mean_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/8_14_originals_working/2024-07-31_06-34-03_job_number_77.h5ad')
mean_adata = mean_adata[mean_adata.obs['type'] == 'reprogrammed']
#cluster 9 of 61 seems to do the worst in recipe_diff and reprog_to_hsc.

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

datasets = ['max_61_7', 'mean_77_2', 'fib_NaN_NaN', 'hsc_hsc1_NaN']
centroids_high_dim = {dataset: calculate_centroids(plot_adata, dataset) for dataset in datasets}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array(list(centroids_high_dim.values()))
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Plot the UMAP
plt.figure(figsize=(12, 8))
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette='RdBu', alpha=0.6)

# Add centroid markers
centroid_labels = datasets
for i, dataset in enumerate(datasets):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], marker='X', s=100, label=f'Centroid {dataset}', edgecolor='black', color='red')

# 4. Draw Lines Between Centroids and Compute Cosine Distances
distances = cosine_distances(centroids_high_dim_matrix)
for i, start in enumerate(datasets):
    for j, end in enumerate(datasets):
        if i < j:
            plt.plot([centroids_umap[i, 0], centroids_umap[j, 0]], [centroids_umap[i, 1], centroids_umap[j, 1]], 'k--', alpha=0.6)
            mid_point = [(centroids_umap[i, 0] + centroids_umap[j, 0]) / 2, (centroids_umap[i, 1] + centroids_umap[j, 1]) / 2]
            plt.text(mid_point[0], mid_point[1], f'{distances[i, j]:.2f}', color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances
import anndata as ad

# Load the HSC data
hsc_adata = ad.read_h5ad('/nfs/turbo/umms-indikar/shared/projects/geneformer/fib15k/originals/hsc_1_use_this_copy.h5ad')

# Initialize a list to store the centroids of each kmeans cluster
kmeans_centroids = []

# Calculate the centroid for each cluster
for cluster in range(10):  # Assuming there are 10 clusters labeled from 0 to 9
    cluster_data = max_min_adata[max_min_adata.obs['kmeans_clusters'] == cluster].X
    centroid = np.mean(cluster_data, axis=0)
    kmeans_centroids.append(centroid)

# Calculate the centroid of the HSC data
hsc_centroid = np.mean(hsc_adata.X, axis=0)

# Compute pairwise cosine distances between each K-means centroid and the HSC centroid
distances_to_hsc = cosine_distances(kmeans_centroids, hsc_centroid.reshape(1, -1))

# Print the distances
for i, distance in enumerate(distances_to_hsc):
    print(f"Distance from K-means cluster {i} to HSC centroid: {distance[0]}")


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'max_61_7': centroid_max,
    'min_61_9': centroid_min,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each dataset
centroid_colors = {
    'max_61_7': 'blue',  # Example color
    'min_61_9': 'orange',  # Example color
    'mean_77_2': '#E41A1C',
    'fib_NaN_NaN': '#377EB8',
    'hsc_hsc1_NaN': '#FF7F00',
    'max_min_combined': 'green'
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array(list(centroids_high_dim.values()))
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Plot the UMAP
plt.figure(figsize=(12, 8))

# Plot the UMAP scatter plot with the updated palette
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette=centroid_colors, alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=100, label=f'Centroid {dataset}', 
                edgecolor='black', color=centroid_colors[dataset])

# 4. Draw Lines Between Centroids and Compute Cosine Distances
distances = cosine_distances(centroids_high_dim_matrix)
for i, start in enumerate(centroid_labels):
    for j, end in enumerate(centroid_labels):
        if i < j:
            plt.plot([centroids_umap[i, 0], centroids_umap[j, 0]], 
                     [centroids_umap[i, 1], centroids_umap[j, 1]], 
                     'k--', alpha=0.6)
            mid_point = [(centroids_umap[i, 0] + centroids_umap[j, 0]) / 2, 
                         (centroids_umap[i, 1] + centroids_umap[j, 1]) / 2]
            plt.text(mid_point[0], mid_point[1], f'{distances[i, j]:.2f}', 
                     color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'max_61_7': centroid_max,
    'min_61_9': centroid_min,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each dataset
centroid_colors = {
    'max_61_7': '#C93784',  # color 5
    'min_61_9': '#0077CC',  # COLOR 1
    'mean_77_2': '#A23A8E', # color 4
    'fib_NaN_NaN': '#4C52A5', # COLOR 2
    'hsc_hsc1_NaN': '#E34390', #color 6 - pinkest
    'max_min_combined': '#7D3F98' # color 3
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array([centroid_combined, centroid_max, centroid_min, centroid_mean, centroid_fib, centroid_hsc])
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Calculate and Display Only the Distances to HSC Centroid
# Get the index of the HSC centroid
hsc_index = list(centroids_high_dim.keys()).index('hsc_hsc1_NaN')

# Compute cosine distances from all other centroids to the HSC centroid
distances_to_hsc = cosine_distances(centroids_high_dim_matrix, centroids_high_dim_matrix[hsc_index:hsc_index+1])

# 4. Plot the UMAP
plt.figure(figsize=(12, 8))

# Plot the UMAP scatter plot with the updated palette
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette=centroid_colors, alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=100, label=f'Centroid {dataset}', 
                edgecolor='black', color=centroid_colors[dataset])

# Draw lines and display distances only between centroids and HSC centroid
for i, dataset in enumerate(centroid_labels):
    if i != hsc_index:
        plt.plot([centroids_umap[i, 0], centroids_umap[hsc_index, 0]], 
                 [centroids_umap[i, 1], centroids_umap[hsc_index, 1]], 
                 'k--', alpha=0.6)
        mid_point = [(centroids_umap[i, 0] + centroids_umap[hsc_index, 0]) / 2, 
                     (centroids_umap[i, 1] + centroids_umap[hsc_index, 1]) / 2]
        plt.text(mid_point[0], mid_point[1], f'{distances_to_hsc[i, 0]:.2f}', 
                 color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances to HSC Centroid')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'max_61_7': centroid_max,
    'min_61_9': centroid_min,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each dataset
centroid_colors = {
    'max_61_7': '#C93784',  # color 5
    'min_61_9': '#0077CC',  # COLOR 1
    'mean_77_2': '#A23A8E', # color 4
    'fib_NaN_NaN': '#4C52A5', # COLOR 2
    'hsc_hsc1_NaN': '#E34390', #color 6 - pinkest
    'max_min_combined': '#7D3F98' # color 3
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array([centroid_combined, centroid_max, centroid_min, centroid_mean, centroid_fib, centroid_hsc])
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Calculate and Display Only the Distances to HSC Centroid
# Get the index of the HSC centroid
hsc_index = list(centroids_high_dim.keys()).index('hsc_hsc1_NaN')

# Compute cosine distances from all other centroids to the HSC centroid
distances_to_hsc = cosine_distances(centroids_high_dim_matrix, centroids_high_dim_matrix[hsc_index:hsc_index+1])

# 4. Plot the UMAP
plt.figure(figsize=(12, 8))

# Plot the UMAP scatter plot with the updated palette
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette=centroid_colors, alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=200, label=f'Centroid {dataset}',  # Increased marker size
                edgecolor='black', color=centroid_colors[dataset])

# Draw lines and display distances only between centroids and HSC centroid
for i, dataset in enumerate(centroid_labels):
    if i != hsc_index:
        plt.plot([centroids_umap[i, 0], centroids_umap[hsc_index, 0]], 
                 [centroids_umap[i, 1], centroids_umap[hsc_index, 1]], 
                 'k--', alpha=0.6)
        mid_point = [(centroids_umap[i, 0] + centroids_umap[hsc_index, 0]) / 2, 
                     (centroids_umap[i, 1] + centroids_umap[hsc_index, 1]) / 2]
        plt.text(mid_point[0], mid_point[1], f'{distances_to_hsc[i, 0]:.2f}', 
                 color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances to HSC Centroid')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'max_61_7': centroid_max,
    'min_61_9': centroid_min,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each dataset
centroid_colors = {
    'max_61_7': '#C93784',  # color 5
    'min_61_9': '#0077CC',  # COLOR 1
    'mean_77_2': '#A23A8E', # color 4
    'fib_NaN_NaN': '#4C52A5', # COLOR 2
    'hsc_hsc1_NaN': '#E34390', #color 6 - pinkest
    'max_min_combined': '#7D3F98' # color 3
}

# Define a dictionary for renaming in the legend
legend_labels = {
    'max_61_7': 'Max 61 7',
    'min_61_9': 'Min 61 9',
    'mean_77_2': 'Mean 77 2',
    'fib_NaN_NaN': 'Fib NaN NaN',
    'hsc_hsc1_NaN': 'HSC HSC1 NaN',
    'max_min_combined': 'Max & Min Combined'
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array([centroid_combined, centroid_max, centroid_min, centroid_mean, centroid_fib, centroid_hsc])
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Calculate and Display Only the Distances to HSC Centroid
# Get the index of the HSC centroid
hsc_index = list(centroids_high_dim.keys()).index('hsc_hsc1_NaN')

# Compute cosine distances from all other centroids to the HSC centroid
distances_to_hsc = cosine_distances(centroids_high_dim_matrix, centroids_high_dim_matrix[hsc_index:hsc_index+1])

# 4. Plot the UMAP
plt.figure(figsize=(12, 8))

# Plot the UMAP scatter plot with the updated palette
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette=centroid_colors, alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=200, label=legend_labels[dataset],  # Increased marker size and renamed label
                edgecolor='black', color=centroid_colors[dataset])

# Draw lines and display distances only between centroids and HSC centroid
for i, dataset in enumerate(centroid_labels):
    if i != hsc_index:
        plt.plot([centroids_umap[i, 0], centroids_umap[hsc_index, 0]], 
                 [centroids_umap[i, 1], centroids_umap[hsc_index, 1]], 
                 'k--', alpha=0.6)
        mid_point = [(centroids_umap[i, 0] + centroids_umap[hsc_index, 0]) / 2, 
                     (centroids_umap[i, 1] + centroids_umap[hsc_index, 1]) / 2]
        plt.text(mid_point[0], mid_point[1], f'{distances_to_hsc[i, 0]:.2f}', 
                 color='black', ha='center')

# Add legend
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances to HSC Centroid')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()


In [ ]:
## FIxing the overlap to read lines more easily

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics.pairwise import cosine_distances
import umap

# Ensure no duplicate indices in plot_adata.obs
plot_adata.obs = plot_adata.obs.reset_index(drop=True)

# 1. Calculate Centroids in High-Dimensional Space
def calculate_centroids(adata, dataset_name):
    data = adata[adata.obs['dataset'] == dataset_name].X
    return np.mean(data, axis=0)

# Calculate centroids for individual datasets
centroid_max = calculate_centroids(plot_adata, 'max_61_7')
centroid_min = calculate_centroids(plot_adata, 'min_61_9')

# Combine the centroids for max_61_7 and min_61_9
centroid_combined = np.mean([centroid_max, centroid_min], axis=0)

# Calculate centroids for other datasets
centroid_mean = calculate_centroids(plot_adata, 'mean_77_2')
centroid_fib = calculate_centroids(plot_adata, 'fib_NaN_NaN')
centroid_hsc = calculate_centroids(plot_adata, 'hsc_hsc1_NaN')

# Create a dictionary of centroids
centroids_high_dim = {
    'max_min_combined': centroid_combined,
    'max_61_7': centroid_max,
    'min_61_9': centroid_min,
    'mean_77_2': centroid_mean,
    'fib_NaN_NaN': centroid_fib,
    'hsc_hsc1_NaN': centroid_hsc
}

# Define colors for each dataset
centroid_colors = {
    'max_61_7': '#C93784',  # color 5
    'min_61_9': '#0077CC',  # COLOR 1
    'mean_77_2': '#A23A8E', # color 4
    'fib_NaN_NaN': '#4C52A5', # COLOR 2
    'hsc_hsc1_NaN': '#E34390', #color 6 - pinkest
    'max_min_combined': '#7D3F98' # color 3
}

# 2. Transform Centroids to UMAP Space
# Fit UMAP on the entire dataset to get the same UMAP transformation
umap_model = umap.UMAP()
umap_embedding = umap_model.fit_transform(plot_adata.X)

# Create a DataFrame for UMAP plot
umap_df = pd.DataFrame(umap_embedding, columns=['UMAP1', 'UMAP2'])
umap_df['dataset'] = plot_adata.obs['dataset']

# Project centroids to UMAP space
centroids_high_dim_matrix = np.array([centroid_combined, centroid_max, centroid_min, centroid_mean, centroid_fib, centroid_hsc])
centroids_umap = umap_model.transform(centroids_high_dim_matrix)

# 3. Calculate and Display Only the Distances to HSC Centroid
# Get the index of the HSC centroid
hsc_index = list(centroids_high_dim.keys()).index('hsc_hsc1_NaN')

# Compute cosine distances from all other centroids to the HSC centroid
distances_to_hsc = cosine_distances(centroids_high_dim_matrix, centroids_high_dim_matrix[hsc_index:hsc_index+1])

# 4. Plot the UMAP
plt.figure(figsize=(12, 8))

# Plot the UMAP scatter plot with the updated palette
sns.scatterplot(data=umap_df, x='UMAP1', y='UMAP2', hue='dataset', palette=centroid_colors, alpha=0.6)

# Add centroid markers with specific colors
centroid_labels = list(centroids_high_dim.keys())
for i, dataset in enumerate(centroid_labels):
    plt.scatter(centroids_umap[i, 0], centroids_umap[i, 1], 
                marker='X', s=200, label=f'Centroid {dataset}',  # Increased marker size
                edgecolor='black', color=centroid_colors[dataset])

# Draw lines and display distances only between centroids and HSC centroid
for i, dataset in enumerate(centroid_labels):
    if i != hsc_index:
        plt.plot([centroids_umap[i, 0], centroids_umap[hsc_index, 0]], 
                 [centroids_umap[i, 1], centroids_umap[hsc_index, 1]], 
                 'k--', alpha=0.6)
        mid_point = [(centroids_umap[i, 0] + centroids_umap[hsc_index, 0]) / 2, 
                     (centroids_umap[i, 1] + centroids_umap[hsc_index, 1]) / 2]
        plt.text(mid_point[0], mid_point[1], f'{distances_to_hsc[i, 0]:.2f}', 
                 color='black', ha='center')

# Add legend
plt.legend(loc='best')

# Set titles and labels
plt.title('UMAP with Centroids and Cosine Distances to HSC Centroid')
plt.xlabel('UMAP1')
plt.ylabel('UMAP2')

plt.show()
